### Assigning variables

```bash
foo=bar
```
Then we can do

```bash
echo $foo
```
to display it. Notice that we don't use blank space for variable assignment. If we did

```bash
foo = bar
```
it will throw an error because then it will try to run the command foo with arguments = and bar. And obviously command foo is not found. Spaces are also important when working with strings. For strings, we can use either single quotes or double quotes

```bash
echo "hello"
echo 'world'
```
and it will work fine, however, if we deal with variables, we should use double quotes as they will expand the variable

```bash
echo "Value is $foo"
>Value is bar

echo 'Value is $foo'
>Value is $foo
```

We've seen chaining commands but a lot of times we want to do one thing first and then another thing.

Contents of ---mch.sh---

```bash
mcd () {
    mkdir -p "$1"
    cd "$1"
}

```
now we can do

```bash
source mcd.sh
```
and it will look like nothing happened, but it will execute and load mcd for us to use

```bash
mcd test_folder
pwd
>Users/dipamvasani/Desktop/test_folder
```
The source command can be used to load any functions file into the current shell script or a command prompt.

```bash
$0 - name of script
$1 to $9 - arguments
$? - error code of the last command
$_ - last argument of the previous command
$# - number of arguments
$$ - process id of the command that is running
$@ - all the arguments
```
For example:

```bash
mkdir test_folder
cd $_

```

Another useful shortcut is !!. If you run a command and you don't have permission for it

```bash
mkdir sbin/my_folder
Permission denied

sudo !!
```
and it'll replace !! with the previous command that was run.

Just like standard input and standard output, there's standard error stream for bash. It's with the use of error codes. It represents how the run of the program went. If we do something like

```bash
echo "hello"

echo $?
>0
```
the output is 0 because things went okay, no errors. If we search foobar in the file mcd.sh

```bash
echo foobar mcd.sh
echo $?
>1
```
true will always have 0 error code and false will always have 1 error code.
```bash
false || echo "Oops fail"
>Oops fail
```
It will run the first one and if it doesn't return 0 then it'll run the second one. Otherwise it won't run the second one.

```bash
true || echo "This will not run"
```
Similarly there's logical and

```bash
false && echo "This will not print"
true && echo "Oops fail"
>Oops fail
```
You can concatenate commands using semicolon

```bash
false ; echo "This will always print"
```
If we want to store the output of a command in a variable then we

```bash
foo=$(pwd)
echo $foo
>/Users/dipamvasani/Desktop
```
We can do it directly as well
```bash
echo "We are in $(pwd)"
```
Concatenate the output of 2 ls commands

```bash
cat <(ls) <(ls ..)
```
Let's try an example

```bash
#!/bin/bash

# print the date
echo "Starting program at $(date)"

# print some stuff
echo "Running program $0 with $# arguments and pid $$"

# for file in arguments
for file in "$@"; do

        # check if the file has foobar
        # we are just interested in the error code hence we redirect the output
        # and the standard error (2>) to a file called /dev/null
        # we can dump as many things in this file as we want and our system will empty it
        grep foobar "$file" > /dev/null 2> /dev/null
        
        # we then check if the error code is not equal to 0
        # to find out about more comparisons check man test
        if [[ "$?" -ne 0 ]]; then
                echo "File $file does not have foobar, adding one"
                echo "# foobar" >> "$file"
        fi
done
```
After giving it the necessary permissions we can run it


```bash
./example.sh mcd.sh practice/beeraj.py practice/test.py practice/foo.txt 
>Starting program at Sat Mar 28 20:35:14 EDT 2020
>Running program ./example.sh with 4 arguments and pid 2858
>File mcd.sh does not have foobar, adding one
>File practice/beeraj.py does not have foobar, adding one
>File practice/test.py does not have foobar, adding one
>File practice/foo.txt does not have foobar, adding one

```
if we run it once more

```bash
./example.sh mcd.sh
```
no output, because we already added foobar. Btw, we can also feed the script to itself. Usually there will be more succint ways to give the filenames to the program like
```bash
ls *.sh | ./example.sh
```
for passing all sh files. * is for 0 or more characters. If we want just one character we use question mark

```bash
mkdir project4
mkdir project5
mkdir project32
ls project?
>project4:
>
>project5:
```
If our arguments are similar, we can do something like this

```bash
convert image.png image.jpg
# instead we can also do
convert image.{png,jpg}
```
This is really powerful, if we want to create a bunch of files we can do

```bash
touch foo{,2,3,10}
```
Suppose we want to create multiple files and folders

```bash
mkdir -p Indiana/{mlsp,cv,eda}/assignments/assignment1/
touch Indiana/{mlsp,cv,eda}/assignments/assignment1/{readme,contributing}.md
```
We can also do
```bash
mkdir foo bar
touch {foo,bar}/{a..j}
```
now suppose we create 2 more files, and we want to see the difference between the files in the 2 folders

```bash
touch foo/x bar/y
diff <(ls foo) <(ls bar)
output:
11c11
< x
---
> y
```

We can interact with the shell using other languages as well, and sometimes other languages are good for the task we want to achieve. For example, Python. Python does not have the sys library as default so we import it.
Contents of script.py

```python
#!/usr/local/bin/python3
import sys
for arg in reversed(sys.argv[1:]):
    print(arg)

```
And then we can run it as python3 script.py. The first line is called shebang and is used to indicate that we want to run this script using python. Hence, after giving it permissions, we can also run it as a script.
```bash
./script.py a b c
c
b
a

```
Now one thing we want to do is not assume where python or any other interpreter is installed, hence we can use the env command which is in pretty much every system with an argument to specify python

```bash
#!/usr/bin/env python3
import sys
for arg in reversed(sys.argv[1:]):
    print(arg)

```
This will use the path variable to find python and then use that to run the script. Bash can be difficult to debug sometimes, so there is this simple tool called shellcheck that gives potential problems with your script and suggestions to improve it

```bash
shellcheck mcd.sh

In mcd.sh line 1:
mcd () {
^-- SC2148: Tips depend on target shell and yours is unknown. Add a shebang.


In mcd.sh line 3:
	cd "$1"
        ^-----^ SC2164: Use 'cd ... || exit' or 'cd ... || return' in case cd fails.

Did you mean: 
	cd "$1" || exit

For more information:
  https://www.shellcheck.net/wiki/SC2148 -- Tips depend on target shell and y...
  https://www.shellcheck.net/wiki/SC2164 -- Use 'cd ... || exit' or 'cd ... |...

```
The way we find out the flags and options with a command is using the man page. However, sometimes the man pages are enormous and hard to read. For such cases we can use a tool like tldr to give use nice, concise examples of how to use the command

```bash
tldr ls


ls

List directory contents.

- List files one per line:
    ls -1

- List all files, including hidden files:
    ls -a

- Long format list (permissions, ownership, size and modification date) of all files:
    ls -la

- Long format list with size displayed using human readable units (KB, MB, GB):
    ls -lh

- Long format list sorted by size (descending):
    ls -lS

- Long format list of all files, sorted by modification date (oldest first):
    ls -ltr

```

To find files in a directory we can use find

```bash
find .  -iname assignments -type d
```
find files in the current folder (dot) with the name assignments (iname means ignore case) and type folder. Find has a lot of useful flags like pattern matching 

```bash
find . -path "**/IU/*.py" -type f
```
Find all the files in the current path, pattern = some number of folders / IU / python files and check type file.

Things modified in the last day

```bash
find . -mtime 1
```
and you can use other things like size, owner, permission, you name it. Find does not only find files, it can also execute stuff. For example, remove all files with the following pattern

```bash
find . -name "*.tmp" -exec rm {} \;
```
Now we can run a lot of finds during the day, so wouldn't it be better for our system to maintain a databse of sorts, with indexes to find files easily, and then update it every time we do new finds. This is done in most sytems using the locate command. This will look for patterns matching

```bash
locate missing-semester
```
use the updatedb command to update this database

```bash

```


```bash

```

```bash

```

```bash

```

```bash

```

```bash

```


```bash

```

```bash

```

```bash

```

```bash

```

```bash

```


```bash

```

```bash

```

```bash

```

```bash

```



